# Model building

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd

import seaborn as sns

from tools.helper_functions import moving_average, normalize, rolling_normal
from tools.data_loader import load_flight_data, select_sequence, clean_data, apply_motor_calibration

import tools.data_loader as dl

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

from scipy.signal import savgol_filter
from scipy.interpolate import interp1d
from scipy.integrate import cumulative_trapezoid, simpson, romb
        
import glob
import os

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# declare ALL list objects necessary

v_induced = 14.345219306215128 # induced velocity for firefly vehicle
markersize= 15 # global markersize setting for scatter plots

# set colors

lowCol = '#FF7518' # pumpkin orange
upCol = '#18A2FF' # complementary color

upMeanCol = 'blue'
lowMeanCol = 'black'

armCol = upCol
armMeanCol = 'red'

## Load flight data

In [ ]:
# select test flight

# first testflight
#flight = f'../flight_data/2022-08-01_ag_field/flight_6_first_hover_flight/'
#hover_start = 35
#hover_end = 15

# second testflight
flight = f'../flight_data/2022-08-01_ag_field/flight_8_second_hover_flight/'
hoverStart = 30
hoverEnd = 30

# third testflight
#flight = f'../flight_data/2022-08-29_ag_field/flight_2_delta0_sweep/'
#hover_start = 30
#hover_end = 30

In [ ]:
fd = load_flight_data(flight)
fd = dl.convert_time(fd)

## Preprocessing

### Rename columns and calculate power values for motors, arms and vehicle

In [ ]:
fd = clean_data(fd)

### Apply calibration for ESC current and voltage

In [ ]:
fd = apply_motor_calibration(fd)

In [ ]:
# Perform regression RPM vs. current

rpm_range = np.linspace(0, 3000, 100).reshape(-1,1)
currentRegMotorValues = []
currentRegCurve = []
fd_current = []


for i in range(1,9):
    
    # get all points where I3 has its minimum value and RPM is greater than 0
    filtered = fd[fd[f'I{i}']> fd[f'I{i}'].min()]
    filtered = filtered[filtered[f'rpm{i}']> 0]
    
    rpm = filtered[[f'rpm{i}']]
    current = filtered[[f'I{i}']]
    
    rpmAsPolynomial = PolynomialFeatures(degree=3, include_bias=False).fit_transform(rpm)
    
    currentRegression = LinearRegression(fit_intercept=False, positive=True).fit(rpmAsPolynomial, current)

    rpmRangePolynomial = PolynomialFeatures(degree=3, include_bias=False).fit_transform(rpm_range)
    
    currentPredictions = currentRegression.predict(rpmRangePolynomial)
    
    currentRegMotorValues.append(currentPredictions)
    fd_current.append(filtered)
    currentRegCurve.append(currentRegression)

In [ ]:
# correct values for current by using regression model

# save value of flight data DataFrame
#fd_corrected = fd.copy()
fd_old = fd.copy(deep=True)

# get indices where I3 is minimum
minI3 = fd['I3']==fd[f'I3'].min()
rpm3Data = fd['rpm3'][minI3].values.reshape(-1,1)

# transform polynomial features
rpm3Features = PolynomialFeatures(degree=3, include_bias=False).fit_transform(rpm3Data)

# perform regression
rpm3RegressionResult = currentRegCurve[2].predict(rpm3Features).reshape(-1)

# apply correction to current signal
fd.loc[minI3,'I3'] = rpm3RegressionResult

### Calculate power and controls

In [ ]:
fd = dl.calculate_power_and_rpm(fd)

### Calculate motor commands

In [ ]:
fd = dl.calculate_motor_cmds(fd)

### Filter out hover sequence DataFrame

In [ ]:
# Reset current correction
hoverIndex, fd_hover = select_sequence(fd, 30, 30)

### Calculate mean and median values for each 

In [ ]:
delta0Mean = pd.pivot_table(fd_hover, index='delta0', aggfunc=np.mean)
delta0Median = pd.pivot_table(fd_hover, index='delta0', aggfunc=np.median)

### Calculate mean and median values for each 

In [ ]:
delta0Mean = pd.pivot_table(fd_hover, index='delta0', aggfunc=np.mean)
delta0Median = pd.pivot_table(fd_hover, index='delta0', aggfunc=np.median)

## Signal filtering

### Reset time scale

In [ ]:
fd_hover['t'] = np.arange(0,len(fd_hover))*(1/30) 
fd_hover['t'] = fd_hover['t'].round(4)
fd_hover['dt'] = fd_hover['t'].diff().round(3)

### Filter position signal

In [ ]:
fdMean = fd_hover.rolling(30,1).mean().dropna()
fdMean5 = fd_hover.rolling(1,1).mean().dropna()
#fdPosition = fd_hover.transform(lambda x: savgol_filter(x, 11,3))

fdInterp = fd_hover.copy(deep=True)

f_x = interp1d(fdInterp['t'], fdInterp['x'])
fdInterp['x'] = f_x(fdInterp['t'])


In [ ]:
fdPosition = fd_hover.copy(deep=True)
#fdPosition = fdPosition.rolling(10).mean()

# derive velocity from position
fdPosition['dx'] = -fdPosition['x'].diff() / fdPosition['dt']
fdPosition['dy'] = -fdPosition['y'].diff() / fdPosition['dt']

fdPosition = fdPosition.dropna()

# integrate position from velocity
fdPosition['dxdt'] = (fdPosition['x'].iloc[0] - fdPosition['u'].cumsum()*fdPosition['dt'])
fdPosition['dydt'] = (fdPosition['x'].iloc[0] -
                      simpson(y=fdPosition['u'], x=fdPosition['t']))
#fdPosition['dydt'] = (fdPosition['y'].iloc[0] - fdPosition['v'].cumsum()*fdPosition['dt'])

#fdPosition['dxdt'] = 
#print(len(cumtrapz(y=fdPosition['u'], x=fdPosition['t'], initial=fdPosition['x'].iloc[0])))
#print(len(fdPosition))
#fdPosition[['x','dx','dxdt', 'dydt', 'dt']]

In [ ]:
fdPosition['pVehicle']#.iloc[a:b:step]

In [ ]:
# Total power over time

fig0, ax0 = plt.subplots(3,2, figsize=(10,12))
fig0.suptitle('Comparison of power and velocity signal', fontsize=14)

# Rotor pairs
a = 600
b = 1000
step = 1

cm = 'viridis'

# x-y position
ax0[0][0].set_title(f'Projected position')
ax0[0][0].scatter(fd_hover['x'].iloc[a:b:step], fd_hover['y'].iloc[a:b:step], c=fd_hover['t'].iloc[a:b:step],
                  cmap=cm, label='position', s=5)
#ax0[0][0].scatter(fdPosition['x'].iloc[a:b], fdPosition['y'].iloc[a:b], label='position (smoothed)', s=5)

# x position
ax0[0][1].set_title(f'Position over time')


ax0[0][1].scatter(fdPosition['t'].iloc[a:b:step], fdPosition['x'].iloc[a:b:step], c=fdPosition['t'].iloc[a:b:step],
                  cmap=cm, label='position raw', s=5)

ax0[0][1].scatter(fdPosition['t'].iloc[a:b:step], fdPosition['dxdt'].iloc[a:b:step],# c=fdPosition['t'].iloc[a:b:step],
                  cmap='red', label='Position integrated', s=5)

# x-position
ax0[1][0].set_title(f'x-Position over time')
ax0[1][0].plot(fdPosition['t'].iloc[a:b:step], fdPosition['x'].iloc[a:b:step]-fdPosition['x'].mean(), label='x-Signal')

#ax0[1][0].plot(fdPosition['t'].iloc[a:b:step], fdPosition['dxdt'].iloc[a:b:step], label='Position integrated')

# y-position
ax0[1][1].set_title(f'y-Position over time')
ax0[1][0].plot(fdPosition['t'].iloc[a:b:step], fdPosition['y'].iloc[a:b:step]-fdPosition['y'].mean(), label='y-Signal')

ax0[1][1].plot(fdPosition['t'].iloc[a:b:step], fdPosition['pVehicle'].iloc[a:b:step], label='Position integrated')

# x- velocity 
ax0[2][0].set_title(f'u over time')
ax0[2][0].plot(fdPosition['t'].iloc[a:b:step], fdPosition['u'].iloc[a:b:step], label='u')
ax0[2][0].plot(fdPosition['t'].iloc[a:b:step], fdPosition['dx'].iloc[a:b:step], label='u (derived)')


# y velocity
ax0[2][1].set_title(f'v over time')
ax0[2][1].plot(fdPosition['t'].iloc[a:b:step], fdPosition['v'].iloc[a:b:step], label='v')
ax0[2][1].plot(fdPosition['t'].iloc[a:b:step], fdPosition['dy'].iloc[a:b:step], label='v (derived)')

# compare smoothed velocity signals
#ax0[2][0].set_title(f'z-normalized signals')
#ax0[2][0].plot(fd_hover['t'].iloc[a:b], fd_hover['v'].iloc[a:b], label='v')
#ax0[2][0].plot(fdPosition['t'].iloc[a:b], fdPosition['v'].iloc[a:b], label='v (smoothed)')

# Corrected signal
#ax0[2][1].set_title(f'z-normalized signals')
#ax0[2][1].plot(fd_hover['t'].iloc[a:b], fd_hover['w'].iloc[a:b], label='w')
#ax0[2][1].plot(fdPosition['t'].iloc[a:b], fdPosition['w'].iloc[a:b], label='w (smoothed)')



for idx in range(6):
    
    i = int(idx/2)
    j = idx%2
    print(f'i={i}, j={j}')
    
    ax0[i][j].grid()
    ax0[i][j].set_xlabel('Time [s]')
    ax0[i][j].legend()

ax0[0][0].set_ylabel('Velocity [m/s]')
ax0[0][1].set_ylabel('Power [W]')
ax0[1][0].set_ylabel('[-]')
ax0[1][1].set_ylabel('[-]')

fig0.tight_layout()

In [ ]:
df_corr = fdPosition.drop(columns={'t', 'dt'})
df_corr = df_corr.dropna()
df_corr = df_corr.corr()

In [ ]:
df_corr['pVehicle'].abs().sort_values(ascending=False).tail(60)